<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / Time and Timer-Interrupts
***
# Time
## Einführung
**Aufgabe 1:** Eine LED soll fünf mal blinken: jeweils $1,5 s$ lang an und $1,5 s$ aus. Dies geht z.B. einfach mit

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino
pin.off()

from time import sleep

for iIndex in range(5):
    pin.on()
    sleep(1.5)
    pin.off()
    sleep(1.5)

pin.off()

print( "DONE." )

**Aufgabe 1 erweitert:** Jetzt soll auf dem Display die Zeit bis zum Ende des Programms angezeitg werden!  
Ein Versuch:

In [ ]:
#%serialconnect --port=COM3 # Windows with more than one COM-Port
%serialconnect # Linux / Windows with one COM-Port

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino
pin.off()

from time import sleep

from display import Display

display = Display()
display.setCenter( 5, 31 )
display.clear()

iTimeLeft = 15

for iIndex in range(5):
    pin.on()
    display.clear()
    display.text( "Time left: {:>2} s".format( iTimeLeft ), 0, 0 )
    iTimeLeft-=1
    display.show()
    sleep(1)

    display.clear()
    display.text( "Time left: {:>2} s".format( iTimeLeft ), 0, 0 )
    iTimeLeft-=1
    display.show()
    sleep(0.5)
                 
    pin.off()
    sleep(0.5)
                 
    display.clear()
    display.text( "Time left: {:>2} s".format( iTimeLeft ), 0, 0 )
    iTimeLeft-=1
    display.show()
    sleep(1)
                 
pin.off()
display.clear()
display.show()

print( "DONE." )

Das geht, aber wenn sich die Zeiten ändern: oh weia!  
Was auch **nicht optimal** ist: das Ausführen der Display-Ausgabe verbraucht auch Zeit! Die haben wir noch gar nicht berücksichtigt!

Es muss ein **anderer Ansatz** her!

## Ansatz 1
Es werden die verstrichenen **Zeitdifferenzen** seit dem letzten Update (der LED, des Dsiplays) gemessen und nur dann entsprechend Aktionen ausgeführt.

**Das Prinzip:**

Mit $ms$:

In [ ]:
%serialconnect # --port=COM3
import time

lTimeMillis = 1000

start = time.ticks_ms()
while True:
    if time.ticks_diff(time.ticks_ms(), start) > lTimeMillis:
        start = time.ticks_ms()
        print( "Tick" )

Und mit $\mu s$:

In [ ]:
%serialconnect # --port=COM3
import time

lTimeMicros = 1000000

start = time.ticks_us()
while True:
    if time.ticks_diff(time.ticks_us(), start) > lTimeMicros:
        start = time.ticks_us()
        print( "Tick" )

**Aufgabe 1 neu:**

In [ ]:
%serialconnect # --port=COM3

TIME_LED_TICK_MILLIS     = 1500 # 1500 ms = 1.5 s
TIME_DISPLAY_TICK_MILLIS = 1000 # 1000 ms = 1 s


from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino
pin.off()


from display import Display

display = Display()
display.setCenter( 5, 31 )
display.clear()


from time import ticks_ms, ticks_diff

lTimeLEDStartMillis     = ticks_ms() - TIME_LED_TICK_MILLIS
lTimeDisplayStartMillis = ticks_ms() - TIME_DISPLAY_TICK_MILLIS


iTimeLeft = 15
bLEDIsOn = False

while iTimeLeft > 0:
    # Zeit schon rum?
    if ticks_diff(ticks_ms(), lTimeDisplayStartMillis) > TIME_DISPLAY_TICK_MILLIS:
        lTimeDisplayStartMillis = ticks_ms()
        display.clear()
        display.text( "Time left: {:>2} s".format( iTimeLeft ), 0, 0 )
        iTimeLeft-=1
        display.show()
        
    # Zeit schon rum?
    if ticks_diff(ticks_ms(), lTimeLEDStartMillis) > TIME_LED_TICK_MILLIS:
        lTimeLEDStartMillis = ticks_ms()
        bLEDIsOn = not bLEDIsOn
        pin.on() if bLEDIsOn else pin.off()
        

# Cleaning up:
pin.off()
display.clear()
display.show()

print( "DONE." )


## Ansatz 2: Timer-Interrupts
Mit dem `Timer` aus dem Modul `machine`, also mit Interrupts

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

TIME_LED_TICK_MILLIS     = 1500 # 1500 ms = 1.5 s
TIME_DISPLAY_TICK_MILLIS = 1000 # 1000 ms = 1 s

from machine import Pin
pin = Pin(25, Pin.OUT) # 25 HelTec, 13 Croduino
pin.off()


from display import Display

display = Display()
display.setCenter( 5, 31 )
display.clear()


from machine import Timer


bLEDIsOn = False

# Is called when timer-intetrupt occures:
def ledTimerInterruptHandler(timer):
    global bLEDIsOn
    bLEDIsOn = not bLEDIsOn
    pin.on() if bLEDIsOn else pin.off()
    
timerLED = Timer(1)
timerLED.init(period=TIME_LED_TICK_MILLIS, mode=Timer.PERIODIC, callback=ledTimerInterruptHandler)


bDisplayTick = False

# Is called when timer-intetrupt occures:
def displayTimerInterruptHandler(timer):
    global bDisplayTick
    bDisplayTick = True
    
timerDisplay = Timer(2)
timerDisplay.init(period=TIME_DISPLAY_TICK_MILLIS, mode=Timer.PERIODIC, callback=displayTimerInterruptHandler)

iTimeLeft = 15

while iTimeLeft > 0:
    if bDisplayTick :
        bDisplayTick = False
        display.clear()
        display.text( "Time left: {:>2} s".format( iTimeLeft ), 0, 0 )
        iTimeLeft-=1
        display.show()

timerLED    .deinit()
timerDisplay.deinit()

pin.off()
display.clear()
display.show()

print( "DONE." )


## Sample: Analog-Clock

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine
import display
import math

gSeconds = 1800

def timerInterruptHandler(timer):
    global gSeconds
    gSeconds += 1

timer = machine.Timer(1)
#timer.init( period=1000, mode=machine.Timer.PERIODIC, callback=lambda t:print("Welcome to Microcontrollerslab")) 
timer.init(period=1000, mode=machine.Timer.PERIODIC, callback=timerInterruptHandler)


gSecondsLast = 0
display = display.Display()
display.setCenter( 64, 32 )

try:
    while True:
        if gSeconds != gSecondsLast :
            #state = machine.disable_irq()
            #interruptCounter = interruptCounter-1
            #machine.enable_irq(state)
            gSecondsLast = gSeconds

            display.clear()
            display.circle( 0,0,6)
            for iH in range( 12 ) :
                display.lineP( 0,0,math.pi/6*iH, 30, 32 )
            display.fillCircle( 0,0,4)
            s = gSeconds%60
            m = int(gSeconds/60)%60
            h = int(gSeconds/3600)%24
            display.lineP( 0,0,math.pi/ 6*h+math.pi/360*+math.pi/360*m-math.pi/2, 0, 20 )
            display.lineP( 0,0,math.pi/30*m+math.pi/1800*s-math.pi/2, 0, 25 )
            display.lineP( 0,0,math.pi/30*s-math.pi/2, 0, 32 )
            display.show()
except KeyboardInterrupt:
    timer.deinit()
    
print( "Interrupted by user" )


## Sonstiges

In [ ]:
%serialconnect #--port=COM3 # Windows with more than one COM-Port

import machine

rtc = machine.RTC()
rtc.datetime((2022, 2, 21, 8, 30, 0, 0, 0))
print(rtc.datetime())